<a href="https://colab.research.google.com/github/mottasilvia/UCU-NLP/blob/main/Proyecto%20Final/PLN_Proyecto_Final_BETO_con_clases_agrupadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo BETO CASED con clases agrupadas

# Resultados del Modelo BERT con clases agrupadas
**Evaluación**: {'eval_loss': 0.491072416305542, 'eval_runtime': 24.2149, 'eval_samples_per_second': 57.898, 'eval_steps_per_second': 14.495, 'epoch': 3.0}

### Resultados por Clase

| Clase                     | Precision | Recall | F1-score | Support |
|---------------------------|-----------|--------|----------|---------|
| agradecimiento o saludo   | 0.96      | 0.94   | 0.95     | 461     |
| autorespuesta             | 0.94      | 0.89   | 0.92     | 19      |
| baja                      | 0.88      | 0.67   | 0.76     | 21      |
| equivocado                | 0.83      | 0.83   | 0.83     | 12      |
| glicemia                  | 0.40      | 0.13   | 0.20     | 15      |
| medicación o turno        | 0.57      | 0.47   | 0.52     | 36      |
| optin                     | 0.98      | 0.92   | 0.95     | 385     |
| otros                     | 0.68      | 0.80   | 0.74     | 56      |
| respuesta                 | 0.82      | 0.91   | 0.87     | 397     |
| **accuracy**              |           |        | 0.90     | 1402    |
| **macro avg**             | 0.79      | 0.73   | 0.75     | 1402    |
| **weighted avg**          | 0.90      | 0.90   | 0.89     | 1402    |

### Conclusiones

- **Accuracy**: 0.90
- El modelo BERT CASED muestra un buen rendimiento general con altos valores de precisión, recall y f1-score.
- Las clases con mayor número de ejemplos como "agradecimiento o saludo", "optin", y "respuesta" tienen excelentes valores de precisión y recall.
- Las clases menos frecuentes como "glicemia" y "equivocado" presentan un desafío, pero aún así muestran resultados decentes.
- La capacidad del modelo BERT para comprender el contexto y las relaciones semánticas en el texto contribuye a su alto rendimiento.

### Observaciones

- **Comparado con otros modelos**:
  - **Random Forest**: Aunque tiene un buen desempeño general, no alcanza la precisión y recall del modelo BERT en clases menos frecuentes.
  - **RNN (LSTM) y GRU**: Utilizan embeddings aprendidos del corpus de entrenamiento y muestran un buen rendimiento general, pero no logran superar el desempeño de BERT en términos de precisión y recall en la mayoría de las clases.
  
- **Ventaja de BERT**: La arquitectura de BERT y su capacidad para capturar relaciones contextuales y semánticas detalladas en el texto, junto con el uso de embeddings preentrenados y ajustados durante el fine-tuning, le otorgan una ventaja significativa en tareas de clasificación de texto.

### Recomendaciones

- **Uso de BERT**: Para tareas de clasificación de mensajes, especialmente en dominios con clases desbalanceadas y complejas, BERT se destaca como la mejor opción debido a su capacidad para comprender contextos complejos y relaciones semánticas.
- **Entrenamiento adicional**: Considerar más datos de entrenamiento o técnicas de aumento de datos para mejorar aún más el rendimiento en clases con pocos ejemplos.


In [ ]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 24.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [ ]:
!pip install transformers pandas scikit-learn accelerate emoji



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from google.colab import drive
import emoji

In [ ]:
# Verificar si hay una GPU disponible y establecer el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Imprimir información sobre los dispositivos CUDA
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


Usando dispositivo: cuda
True
1
Tesla T4


In [ ]:
# ejecutar luego de subir archivo excel o tener vinculado Google Drive
import os
from google.colab import drive

if os.path.isfile("/content/mensajes_trabajo_final.xlsx"):
  data = pd.read_excel("/content/mensajes_trabajo_final.xlsx")
else:
  # Montar Google Drive
  drive.mount('/content/drive')

  # Cambia el path al archivo que hayas subido a tu Google Drive
  file_path = '/content/drive/MyDrive/UCU-NLP/mensajes_trabajo_final_limpios.xlsx'

  # Cargar el archivo Excel
  data = pd.read_excel(file_path)

print("Datos cargados")

Mounted at /content/drive
Datos cargados


In [ ]:
# Seleccionar las columnas relevantes
data = data[['recibido', 'clase']]

# Mostrar las primeras filas del archivo
print(data.head())

                                            recibido  \
0  :thumbs_up_medium_skin_tone::heart_hands_mediu...   
1                                           Hola....   
2  :clapping_hands_light_skin_tone::clapping_hand...   
3                                            Gracias   
4  Hola .. si suele..cuando me...desvelo..o de ta...   

                            clase  
0                  agradecimiento  
1                          saludo  
2                  agradecimiento  
3                  agradecimiento  
4  respuesta a consejo o pregunta  


Agrupación de Clases

In [ ]:
# agrupar clases

data.loc[data['clase'] == 'agradecimiento', 'clase'] = 'agradecimiento o saludo'
data.loc[data['clase'] == 'saludo', 'clase'] = 'agradecimiento o saludo'
data.loc[data['clase'] == 'medicación', 'clase'] = 'medicación o turno'
data.loc[data['clase'] == 'turno', 'clase'] = 'medicación o turno'
data.loc[data['clase'] == 'queja', 'clase'] = 'otros'
data.loc[data['clase'] == 'pregunta', 'clase'] = 'otros'
data.loc[data['clase'] == 'respuesta a consejo o pregunta', 'clase'] = 'respuesta'

NameError: name 'data' is not defined

In [ ]:
# Convertir etiquetas de clase a representación numérica (suponiendo que son cadenas)
data['clase'] = data['clase'].astype('category')
data['clases_cat'] = data['clase'].cat.codes

In [ ]:
# Diccionario de traducciones de emojis de inglés a español
emoji_translations = {
    ":grinning_face:": ":cara_sonriendo:",
    ":grinning_face_with_big_eyes:": ":cara_sonriendo_con_ojos_grandes:",
    ":grinning_face_with_smiling_eyes:": ":cara_sonriendo_con_ojos_sonrientes:",
    ":beaming_face_with_smiling_eyes:": ":cara_radiante_con_ojos_sonrientes:",
    ":grinning_squinting_face:": ":cara_sonriendo_con_ojos_cerrados:",
    ":grinning_face_with_sweat:": ":cara_sonriendo_con_sudor:",
    ":rolling_on_the_floor_laughing:": ":riendo_en_el_suelo:",
    ":face_with_tears_of_joy:": ":cara_con_lágrimas_de_alegría:",
    ":slightly_smiling_face:": ":cara_ligeramente_sonriente:",
    ":upside_down_face:": ":cara_al_revés:",
    ":winking_face:": ":cara_guiñando:",
    ":smiling_face_with_smiling_eyes:": ":cara_sonriendo_con_ojos_sonrientes:",
    ":smiling_face_with_halo:": ":cara_sonriendo_con_auréola:",
    ":smiling_face_with_hearts:": ":cara_sonriendo_con_corazones:",
    ":smiling_face_with_heart_eyes:": ":cara_sonriendo_con_ojos_de_corazón:",
    ":star_struck:": ":cara_deslumbrada:",
    ":face_blowing_a_kiss:": ":cara_lanzando_un_beso:",
    ":kissing_face:": ":cara_besando:",
    ":smiling_face:": ":cara_sonriente:",
    ":kissing_face_with_closed_eyes:": ":cara_besando_con_ojos_cerrados:",
    ":kissing_face_with_smiling_eyes:": ":cara_besando_con_ojos_sonrientes:",
    ":face_savoring_food:": ":cara_saboreando_comida:",
    ":face_with_tongue:": ":cara_con_lengua:",
    ":winking_face_with_tongue:": ":cara_guiñando_con_lengua:",
    ":zany_face:": ":cara_loca:",
    ":squinting_face_with_tongue:": ":cara_entrecerrada_con_lengua:",
    ":money_mouth_face:": ":cara_con_boca_de_dinero:",
    ":hugging_face:": ":cara_abrazando:",
    ":face_with_hand_over_mouth:": ":cara_con_mano_sobre_la_boca:",
    ":shushing_face:": ":cara_pidiendo_silencio:",
    ":thinking_face:": ":cara_pensando:",
    ":zipper_mouth_face:": ":cara_con_cierre_en_la_boca:",
    ":face_with_raised_eyebrow:": ":cara_con_ceja_levantada:",
    ":neutral_face:": ":cara_neutra:",
    ":expressionless_face:": ":cara_sin_expresión:",
    ":face_without_mouth:": ":cara_sin_boca:",
    ":smirking_face:": ":cara_sonriente:",
    ":unamused_face:": ":cara_no_divertida:",
    ":face_with_rolling_eyes:": ":cara_con_ojos_en_blanco:",
    ":grimacing_face:": ":cara_haciendo_mueca:",
    ":lying_face:": ":cara_mintiendo:",
    ":relieved_face:": ":cara_aliviada:",
    ":pensive_face:": ":cara_pensativa:",
    ":sleepy_face:": ":cara_somnolienta:",
    ":drooling_face:": ":cara_babeando:",
    ":sleeping_face:": ":cara_durmiendo:",
    ":face_with_medical_mask:": ":cara_con_mascarilla:",
    ":face_with_thermometer:": ":cara_con_termómetro:",
    ":face_with_head_bandage:": ":cara_con_venda_en_la_cabeza:",
    ":nauseated_face:": ":cara_nauseada:",
    ":face_vomiting:": ":cara_vomitando:",
    ":sneezing_face:": ":cara_estornudando:",
    ":hot_face:": ":cara_caliente:",
    ":cold_face:": ":cara_fría:",
    ":woozy_face:": ":cara_mareada:",
    ":dizzy_face:": ":cara_mareada:",
    ":exploding_head:": ":cabeza_explosiva:",
    ":cowboy_hat_face:": ":cara_con_sombrero_de_vaquero:",
    ":partying_face:": ":cara_festiva:",
    ":smiling_face_with_sunglasses:": ":cara_sonriendo_con_gafas_de_sol:",
    ":nerd_face:": ":cara_de_nerd:",
    ":face_with_monocle:": ":cara_con_monóculo:",
    ":confused_face:": ":cara_confundida:",
    ":worried_face:": ":cara_preocupada:",
    ":slightly_frowning_face:": ":cara_ligeramente_fruncida:",
    ":frowning_face:": ":cara_fruncida:",
    ":face_with_open_mouth:": ":cara_con_boca_abierta:",
    ":hushed_face:": ":cara_silenciosa:",
    ":astonished_face:": ":cara_asombrada:",
    ":flushed_face:": ":cara_sonrojada:",
    ":pleading_face:": ":cara_suplicante:",
    ":frowning_face_with_open_mouth:": ":cara_fruncida_con_boca_abierta:",
    ":anguished_face:": ":cara_angustiada:",
    ":fearful_face:": ":cara_temorosa:",
    ":cold_sweat:": ":sudor_frío:",
    ":disappointed_but_relieved_face:": ":cara_decepcionada_pero_aliviada:",
    ":crying_face:": ":cara_llorando:",
    ":loudly_crying_face:": ":cara_llorando_fuertemente:",
    ":face_screaming_in_fear:": ":cara_gritando_de_miedo:",
    ":confounded_face:": ":cara_confundida:",
    ":persevering_face:": ":cara_perseverante:",
    ":disappointed_face:": ":cara_decepcionada:",
    ":sweat:": ":sudor:",
    ":weary_face:": ":cara_cansada:",
    ":tired_face:": ":cara_fatigada:",
    ":yawning_face:": ":cara_bostezando:",
    ":face_with_steam_from_nose:": ":cara_con_vapor_saliendo_de_la_nariz:",
    ":pouting_face:": ":cara_haciendo_mueca:",
    ":angry_face:": ":cara_enojada:",
    ":face_with_symbols_on_mouth:": ":cara_con_símbolos_en_la_boca:",
    ":smiling_face_with_horns:": ":cara_sonriendo_con_cuernos:",
    ":angry_face_with_horns:": ":cara_enojada_con_cuernos:",
    ":skull:": ":calavera:",
    ":skull_and_crossbones:": ":calavera_y_huesos_cruzados:",
    ":pile_of_poo:": ":montón_de_caca:",
    ":clown_face:": ":cara_de_payaso:",
    ":ogre:": ":ogro:",
    ":goblin:": ":duende:",
    ":ghost:": ":fantasma:",
    ":alien:": ":extraterrestre:",
    ":alien_monster:": ":monstruo_alienígena:",
    ":robot:": ":robot:",
    ":grinning_cat:": ":gato_sonriendo:",
    ":grinning_cat_with_smiling_eyes:": ":gato_sonriendo_con_ojos_sonrientes:",
    ":cat_with_tears_of_joy:": ":gato_con_lágrimas_de_alegría:",
    ":smiling_cat_with_heart_eyes:": ":gato_sonriendo_con_ojos_de_corazón:",
    ":cat_with_wry_smile:": ":gato_con_sonrisa_irónica:",
    ":kissing_cat:": ":gato_besando:",
    ":weary_cat:": ":gato_cansado:",
    ":crying_cat:": ":gato_llorando:",
    ":pouting_cat:": ":gato_fruncido:",
    ":thumbsup:": ":pulgar_arriba:",
    "thumbs_up_medium_skin_tone:": ":pulgar_arriba:",
    ":thumbs_up:": ":pulgar_arriba:",
    "heart_hands_medium_skin_tone": ":manos formando un corazón:",
    "clapping_hands_light_skin_tone": ":manos clamando:"


# Puedes seguir agregando más traducciones según sea necesario
}

# Función para convertir emoticones a texto y luego traducirlos al español
def convertir_emoticones(texto):
    if isinstance(texto, str):  # Verifica si el input es una cadena
        texto_emoji = emoji.demojize(texto, delimiters=(":", ":"))
        for en, es in emoji_translations.items():
            texto_emoji = texto_emoji.replace(en, es)
        return texto_emoji
    else:
        return texto


In [ ]:
# Aplicar la conversión de emoticones a la columna 'recibido'
data['recibido'] = data['recibido'].apply(convertir_emoticones)



In [ ]:
data.head()

,recibido,clase,clases_cat
0,::pulgar_arriba:::manos formando un corazón::,agradecimiento o saludo,0
1,Hola....,agradecimiento o saludo,0
2,::manos clamando::::manos clamando:: gracias,agradecimiento o saludo,0
3,Gracias,agradecimiento o saludo,0
4,Hola .. si suele..cuando me...desvelo..o de ta...,respuesta,8


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU está disponible y se usará.")
else:
  device = torch.device("cpu")
  print("GPU no está disponible, se usará la CPU.")


GPU está disponible y se usará.


In [ ]:
# Aplicar la conversión de emoticones a la columna 'recibido'
data['recibido'] = data['recibido'].apply(convertir_emoticones)

# Convertir etiquetas de clase a representación numérica (suponiendo que son cadenas)
data['clase'] = data['clase'].astype('category')
data['clases_cat'] = data['clase'].cat.codes

# Filtrar las filas con etiquetas inválidas
data = data[data['clases_cat'] >= 0]

# Verificar que todas las etiquetas estén en el rango correcto
num_labels = len(data['clase'].unique())
print(f"Numero de etiquetas: {num_labels}")
print(f"Etiquetas unicas: {data['clases_cat'].unique()}")
assert data['clases_cat'].min() >= 0 and data['clases_cat'].max() < num_labels

Numero de etiquetas: 9
Etiquetas unicas: [0 8 2 3 7 6 5 1 4]


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


print(f'Tamaño del conjunto de entrenamiento: {len(train_data)}')
print(f'Tamaño del conjunto de prueba: {len(test_data)}')

Tamaño del conjunto de entrenamiento: 5605
Tamaño del conjunto de prueba: 1402


In [ ]:
# Crear el tokenizador
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [ ]:
# Función para tokenizar y convertir en tensores
def tokenize_data(data):
    # Asegurar que la columna 'recibido' contenga solo cadenas
    data['recibido'] = data['recibido'].astype(str)

    # Tokenizar los datos
    inputs = tokenizer(data['recibido'].tolist(), padding=True, truncation=True, return_tensors="pt")

    # Convertir etiquetas de clase a tensores
    labels = torch.tensor(data['clases_cat'].tolist())
    inputs['labels'] = labels

    return inputs


In [ ]:
# Tokenizar los datos de entrenamiento y prueba
train_inputs = tokenize_data(train_data)
test_inputs = tokenize_data(test_data)

# Calcular los pesos de las clases en la CPU
class_weights = compute_class_weight(class_weight='balanced', classes=data['clases_cat'].unique(), y=train_data['clases_cat'])
class_weights = torch.tensor(class_weights, dtype=torch.float)


In [ ]:
# Definir un Dataset personalizado
class CustomDataset(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        # Removed the .to(device) call as tensors will be moved to device in the Trainer
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

train_dataset = CustomDataset(train_inputs)
test_dataset = CustomDataset(test_inputs)

In [ ]:


# Crear el modelo
model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", num_labels=num_labels)
model.to(device)


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
 # Crear una función de pérdida personalizada para incluir los pesos de las clases
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(device)
        inputs = {k: v.to(device) for k, v in inputs.items() if k != "labels"}
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Crear el objeto Trainer con la función de pérdida personalizada
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Cambiado a evaluation_strategy
    per_device_train_batch_size=4,  # Reducir el tamaño del lote para evitar problemas de memoria
    per_device_eval_batch_size=4,   # Reducir el tamaño del lote para evaluación también
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Entrenar el modelo
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.699000,0.599448
2,0.477700,0.510013
3,0.332900,0.491072


TrainOutput(global_step=4206, training_loss=0.5112206735216432, metrics={'train_runtime': 1650.741, 'train_samples_per_second': 10.186, 'train_steps_per_second': 2.548, 'total_flos': 3534407416967130.0, 'train_loss': 0.5112206735216432, 'epoch': 3.0})

In [ ]:
# Evaluar el modelo
eval_results = trainer.evaluate()
print(f"Resultados de la evaluación: {eval_results}")

# Evaluación detallada con classification_report y confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

# Predecir etiquetas para el conjunto de prueba
y_pred = trainer.predict(test_dataset).predictions.argmax(axis=1)
y_true = test_inputs['labels'].numpy()

# Imprimir el reporte de clasificación
report = classification_report(y_true, y_pred, target_names=data['clase'].cat.categories)
print(report)

# Imprimir la matriz de confusión
cm = confusion_matrix(y_true, y_pred)
print(cm)

Resultados de la evaluación: {'eval_loss': 0.491072416305542, 'eval_runtime': 24.2149, 'eval_samples_per_second': 57.898, 'eval_steps_per_second': 14.495, 'epoch': 3.0}
                         precision    recall  f1-score   support

agradecimiento o saludo       0.96      0.94      0.95       461
          autorespuesta       0.94      0.89      0.92        19
                   baja       0.88      0.67      0.76        21
             equivocado       0.83      0.83      0.83        12
               glicemia       0.40      0.13      0.20        15
     medicación o turno       0.57      0.47      0.52        36
                  optin       0.98      0.92      0.95       385
                  otros       0.68      0.80      0.74        56
              respuesta       0.82      0.91      0.87       397

               accuracy                           0.90      1402
              macro avg       0.79      0.73      0.75      1402
           weighted avg       0.90      0.90     

In [ ]:
# Mostrar y_true y y_pred
print("y_true:", y_true)
print("y_pred:", y_pred)

y_true: [6 8 0 ... 8 8 0]
y_pred: [6 1 0 ... 8 8 0]


In [ ]:
# Mostrar y_true, y_pred y los datos originales
results = pd.DataFrame({
    'Mensaje Original': test_data['recibido'],
    'Etiqueta Real': y_true,
    'Predicción': y_pred,
    'Clase Real': test_data['clase'].cat.categories[y_true],
    'Clase Predicha': test_data['clase'].cat.categories[y_pred]
})

print(results)

                                       Mensaje Original  Etiqueta Real  \
5152                               SÍ quiero recibirlos              6   
3607  Hola dias de atencion de lunes a viernes 9:00 ...              8   
246                                      Muchas gracias              0   
3291                               SÍ quiero recibirlos              6   
5842                               NO quiero recibirlos              2   
...                                                 ...            ...   
3657                               SÍ quiero recibirlos              6   
4180                               SÍ quiero recibirlos              6   
5292                                                Min              8   
1335                Si ago mis controles periódicamente              8   
5214                                  Ok muchas gracias              0   

      Predicción               Clase Real           Clase Predicha  
5152           6                    optin 

In [ ]:
# Guardar el DataFrame en un archivo CSV
if os.path.isfile("/content/mensajes_trabajo_final.xlsx"):
  results.to_csv('/content/resultados_detallados_agrupados.csv', index=False)
  print("Resultados guardados en '/content/resultados_detallados_agrupados.csv'")
else:
  results.to_csv('/content/drive/MyDrive/UCU-NLP/resultados_detallados_agrupados.csv', index=False)
  print("Resultados guardados en '/content/drive/MyDrive/UCU-NLP/resultados_detallados_agrupados.csv'")


Resultados guardados en '/content/drive/MyDrive/UCU-NLP/resultados_detallados_agrupados.csv'
